# Perceptrón aplicado a iris

**Lectura del corpus:** $\;$ comprobamos también que las matrices de datos y etiquetas tienen las filas y columnas que toca

In [ ]:
import numpy as np; from sklearn.datasets import load_iris
iris = load_iris(); X = iris.data.astype(np.float16);
y = iris.target.astype(np.uint).reshape(-1, 1);
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

(150, 4) (150, 1) 
 [[5.1015625  3.5        1.40039062 0.19995117 0.        ]
 [4.8984375  3.         1.40039062 0.19995117 0.        ]
 [4.69921875 3.19921875 1.29980469 0.19995117 0.        ]
 [4.6015625  3.09960938 1.5        0.19995117 0.        ]
 [5.         3.59960938 1.40039062 0.19995117 0.        ]]


**Partición del corpus:** $\;$ Creamos un split de iris con un $20\%$ de datos para test y el resto para entrenamiento (training), barajando previamente los datos de acuerdo con una semilla dada para la generación de números aleatorios. Aquí, como en todo código que incluya aleatoriedad (que requiera generar números aleatorios), conviene fijar dicha semilla para poder reproducir experimentos con exactitud.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)

(120, 4) (30, 4)


**Implementación de Perceptrón:** $\;$ devuelve pesos en notación homogénea, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ también el número de errores e iteraciones ejecutadas

In [ ]:
def perceptron(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break;
    return W, E, k

**Aprendizaje de un clasificador (lineal) con Perceptrón:** $\;$ Perceptrón minimiza el número de errores de entrenamiento (con margen)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{Y}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [ ]:
W, E, k = perceptron(X_train, y_train)
print("Número de iteraciones ejecutadas: ", k)
print("Número de errores de entrenamiento: ", E)
print("Vectores de pesos de las clases (en columnas y en notación homogénea):\n", W);

Número de iteraciones ejecutadas:  200
Número de errores de entrenamiento:  2
Vectores de pesos de las clases (en columnas y en notación homogénea):
 [[  10.           85.         -142.        ]
 [ -49.421875    -68.19140625 -176.47265625]
 [  50.171875     -1.72460938 -181.06445312]
 [-189.91210938  -87.70507812   68.69726562]
 [ -86.40258789 -137.78149414  157.88415527]]


**Cálculo de la tasa de error en test:**
El siguiente conjunto de instrucciones calcula la tasa de error para un Perceptron entrenado con b=0.1 y a=1.0 (factor de aprendizaje o 'a') y K=200 iteraciones

In [ ]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Tasa de error en test: {err_test:.1%}")

Tasa de error en test: 16.7%


In [ ]:
err_test * len(X_test)

5.0

**Ajuste del margen:** $\;$ el siguiente bucle es un experimento que ejecuta 5 veces el algoritmo del Perceptron con 5 valores diferentes de $b$ y valor por defecto $α$=1.0 y K=1000 iteraciones, mostrando el error de entrenamiento para cada valor de $b$.

In [ ]:
for b in (.0, .01, .1, 10, 100):
    W, E, k = perceptron(X_train, y_train, b=b, K=1000)
    print(b, E, k)

0.0 3 1000
0.01 5 1000
0.1 3 1000
10 6 1000
100 6 1000


**Interpretación de resultados:** $\;$ los datos de entrenamiento no parecen linealmente separables; no está claro que un margen mayor que cero pueda mejorar resultados, sobre todo porque solo tenemos $30$ muestras de test; con margen nulo ya hemos visto que se obtiene un error (en test) del $16.7\%$
-

In [ ]:
print('# a b K E k Ete')
for a in (0.1,1.0,10,100,1000,10000):
  for b in (0.0,0.01,0.1,1.0,100,1000):
    for k in (200,500,800,1000):
      W, E, k = perceptron(X_train, y_train, b=b, a=a, K=k)
      X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
      y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
      err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
      print('%8.2f %8.2f %6d %6d %6d %8.3f' %(a, b, k, E,k,err_test))
  print('#------- ------ ---- ---- ---- ----')

# a b K E k Ete
    0.10     0.00    200      6    200    0.133
    0.10     0.00    500      1    500    0.167
    0.10     0.00    800      5    800    0.133
    0.10     0.00   1000      3   1000    0.033
    0.10     0.01    200      2    200    0.167
    0.10     0.01    500      3    500    0.033
    0.10     0.01    800      5    800    0.133
    0.10     0.01   1000      3   1000    0.133
    0.10     0.10    200      8    200    0.100
    0.10     0.10    500      5    500    0.167
    0.10     0.10    800      3    800    0.167
    0.10     0.10   1000      4   1000    0.033
    0.10     1.00    200     10    200    0.033
    0.10     1.00    500      3    500    0.033
    0.10     1.00    800      6    800    0.067
    0.10     1.00   1000      6   1000    0.067
    0.10   100.00    200     29    200    0.000
    0.10   100.00    500     19    500    0.033
    0.10   100.00    800     15    800    0.033
    0.10   100.00   1000     15   1000    0.033
    0.10  1000.00    200

**Ejercicio para realizar en clase:**
---


Escribe el código python necesario para mostrar resultados del error de entrenamiento (E), número de iteraciones empleadas (k) y error de test (err_test) para combinaciones de diferentes valores de:
---

# *   a: (factor de aprendizaje: utiliza valores 0.1, 1.0, 10, 100, 1000, 10000)
# *   b: (margen: utiliza valores 0.0, 0.01, 0.1, 1.0, 100, 1000)
# *   K: (iteraciones: utiliza valores 200, 500, 800 y 1000)


Utiliza la siguientes instrucciones para mostrar la cabecera:
---

`print('#      a        b      K      E      k      Ete');`
---
`print('#-------   ------   ----   ----   ----     ----');`
---

y la siguiente instrucción para mostrar los resultados:

`print('%8.2f %8.2f %6d %6d %6d %8.3f' %(a, b, K, E,k,err_test))`
---





Después de completar la ejecución del bucle, podemos extraer algunas conclusiones. En primer lugar, a simple vista, se puede concluir que el error de prueba disminuye a medida que el factor de aprendizaje disminuye. Esto se evidencia al observar que el error de prueba de 0.00 solo se encuentra para los factores de aprendizaje 0.1, 1.0 y 10. Además, también podemos notar que estos errores tienden a ocurrir en los márgenes más altos y en las iteraciones más elevadas. En resumen, para obtener el menor error de prueba, la combinación ideal sería un factor de aprendizaje mínimo (0.1), un margen máximo (1000) y la iteración más grande (1000).